## **PART 1**

We will use the following text classification dataset and is available on Kaggle and the Link is given below as well


https://www.kaggle.com/c/nlp-getting-started/data?select=test.csv

## **PART 2 and PART 3**

In [ ]:
!pip install tensorflow-text

Import Libraries

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
data.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Data Analysis

In [ ]:
data.groupby('target').describe()

id                            ...                         
         count         mean          std  ...     50%     75%      max
target                                    ...                         
0       4342.0  5276.446338  3157.206802  ...  5243.5  8038.5  10848.0
1       3271.0  5661.608071  3097.094809  ...  5676.0  8252.0  10873.0

[2 rows x 8 columns]

Doing Train & Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size = 0.2, stratify = data['target'])

## **I will create Embeddings using BERT**

Encoder Model = https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4

Preprocess = https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


Here i will be using an Functional Model

In [ ]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'text')


In [ ]:
preprocessed_text = bert_preprocess(text_input)


In [ ]:
outputs =  bert_encoder(preprocessed_text)

In [ ]:
l = tf.keras.layers.Dropout(0.1, name = "dropout")(outputs["pooled_output"])

In [ ]:
l = tf.keras.layers.Dense(1, activation= "sigmoid", name = "output")(l)

In [ ]:
model = tf.keras.Model(inputs = [text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5)


Epoch 1/5
191/191 [==============================] - 68s 320ms/step - loss: 0.6480 - accuracy: 0.6305
Epoch 2/5
191/191 [==============================] - 61s 317ms/step - loss: 0.6047 - accuracy: 0.6739
Epoch 3/5
191/191 [==============================] - 61s 319ms/step - loss: 0.5831 - accuracy: 0.7099
Epoch 4/5
191/191 [==============================] - 61s 319ms/step - loss: 0.5724 - accuracy: 0.7151
Epoch 5/5
191/191 [==============================] - 61s 318ms/step - loss: 0.5619 - accuracy: 0.7250


In [ ]:
model.evaluate(X_test, y_test)

48/48 [==============================] - 16s 315ms/step - loss: 0.5462 - accuracy: 0.7223


[0.546238899230957, 0.7222586870193481]

## **PART 4 BERT**

We can use an pretrained Bert Model which was basically trained on an large dataset and we can further train the model on an smaller dataset and it is basically called as model fine tuning 

In the first step, i will install hugging face transformer library, this library helps us to import transformer based pretrained models, One important thing is to note that BERT and GPT 2 are both transformer based models

In [ ]:
!pip install transformers

Import Libraries

Numpy Library allows us to create an high order multi dimensional array

In [ ]:
import numpy as np


Pandas library is used for data manipulation and analysis and to load the dataset

In [ ]:
import pandas as pd


PyTorch is an open source python library for deep learning 

In [ ]:
import torch


 nn module to help us in creating and training of the neural network.

In [ ]:
import torch.nn as nn


Train, Test & Split

In [ ]:
from sklearn.model_selection import train_test_split


Classification Report 

In [ ]:
from sklearn.metrics import classification_report

Importing  transformers, Transformers provides thousands of pretrained models 


In [ ]:
import transformers


AutoModel is a generic model class that will be instantiated as one of the base model classes of the library when created with the AutoModel. BertTokenizer is a class to help you encode question and paragraph sentence.

In [ ]:
from transformers import AutoModel, BertTokenizerFast

torch. cuda is used to set up and run CUDA operations. It keeps track of the currently selected GPU, and all CUDA tensors you allocate will by default be created on that device. The selected device can be changed with a torch.

In [ ]:
device = torch.device("cuda")

We are using Text Classification Dataset i.e. Natural Language Processing with Disaster Tweets Dataset which is available on Kaggle

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(data['text'], data['target'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=data['target'])

In [ ]:
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

Import BERT-base pretrained model

Load the Bert Tokenizer


In [ ]:
bert = AutoModel.from_pretrained('bert-base-uncased', return_dict=False)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', return_dict=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 25,
    pad_to_max_length=True,
    truncation=True
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:

train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_seq, train_mask, train_y)

train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)


val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT(nn.Module):

    def __init__(self, bert):
      
      super(BERT, self).__init__()

      self.bert = bert 
      
      self.dropout = nn.Dropout(0.1)
      
      self.relu =  nn.ReLU()

      self.fc1 = nn.Linear(768,512)
      
      self.fc2 = nn.Linear(512,2)

      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):

      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      x = self.fc2(x)
      
      x = self.softmax(x)

      return x

model = BERT(bert)

model = model.to(device)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 1e-5)     

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)

print("Class Weights:",class_weights)

Class Weights: [0.87676867 1.16353712]


In [ ]:
weights= torch.tensor(class_weights,dtype=torch.float)

weights = weights.to(device)

cross_entropy  = nn.NLLLoss(weight=weights) 

epochs = 10

In [ ]:
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  total_preds=[]
  
  for step,batch in enumerate(train_dataloader):
    
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    model.zero_grad()        

    preds = model(sent_id, mask)

    loss = cross_entropy(preds, labels)

    total_loss = total_loss + loss.item()

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()

    preds=preds.detach().cpu().numpy()

    total_preds.append(preds)

  avg_loss = total_loss / len(train_dataloader)
  
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds


In [ ]:
def evaluate():
    
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  total_preds = []

  for step,batch in enumerate(val_dataloader):
    
    if step % 50 == 0 and not step == 0:

      elapsed = format_time(time.time() - t0)

      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    with torch.no_grad():

      preds = model(sent_id, mask)

      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  avg_loss = total_loss / len(val_dataloader) 

  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:
best_valid_loss = float('inf')

train_losses=[]
valid_losses=[]

for epoch in range(epochs):

    train_loss, _ = train()
    
    valid_loss, _ = evaluate()
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\Printing the Training Loss: {train_loss:.3f}')
    print(f'Printing the Validation Loss: {valid_loss:.3f}')

  Batch    50  of    167.
  Batch   100  of    167.
  Batch   150  of    167.
\Printing the Training Loss: 0.664
Printing the Validation Loss: 0.654
  Batch    50  of    167.
  Batch   100  of    167.
  Batch   150  of    167.
\Printing the Training Loss: 0.653
Printing the Validation Loss: 0.638
  Batch    50  of    167.
  Batch   100  of    167.
  Batch   150  of    167.
\Printing the Training Loss: 0.640
Printing the Validation Loss: 0.628
  Batch    50  of    167.
  Batch   100  of    167.
  Batch   150  of    167.
\Printing the Training Loss: 0.631
Printing the Validation Loss: 0.622
  Batch    50  of    167.
  Batch   100  of    167.
  Batch   150  of    167.
\Printing the Training Loss: 0.631
Printing the Validation Loss: 0.614
  Batch    50  of    167.
  Batch   100  of    167.
  Batch   150  of    167.
\Printing the Training Loss: 0.621
Printing the Validation Loss: 0.610
  Batch    50  of    167.
  Batch   100  of    167.
  Batch   150  of    167.
\Printing the Training Loss: